In [1]:
import matplotlib.pyplot as plt
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
data = pd.read_json("../data/dump.json")

In [3]:
data.shape

(7949, 19)

In [4]:
data.head()

,action,change_size,flags,geo_ip,hashtags,is_anon,is_bot,is_minor,is_new,is_unpatrolled,mentions,ns,page_title,parent_rev_id,rev_id,summary,timestamp,url,user
0,edit,377.0,None,NaN,[],False,False,False,False,False,[],Talk,Talk:Bajadasaurus,883357329.0,883356516.0,/* Images */,2019-02-14 22:43:57.810307,https://en.wikipedia.org/w/index.php?diff=8833...,FunkMonk
1,edit,-66.0,None,NaN,[],False,False,False,False,False,[],Main,The King of Fighters: Maximum Impact,883357327.0,883220913.0,None,2019-02-14 22:43:58.168588,https://en.wikipedia.org/w/index.php?diff=8833...,Benjaminkirsc
2,edit,-4635.0,B,NaN,[],False,True,False,False,False,[],User,User:AlexNewArtBot/WomensHistorySearchResult,883357322.0,883032645.0,"most recent results, added 113, removed 145, t...",2019-02-14 22:43:58.324169,https://en.wikipedia.org/w/index.php?diff=8833...,InceptionBot
3,edit,1.0,None,NaN,[],False,False,False,False,False,[],Template,Template:Samoa-party-stub,883357331.0,398617982.0,None,2019-02-14 22:43:58.613009,https://en.wikipedia.org/w/index.php?diff=8833...,HapHaxion
4,edit,37.0,None,NaN,[],False,False,False,False,False,[],Main,Persela Lamongan,883357330.0,883030131.0,None,2019-02-14 22:43:58.668958,https://en.wikipedia.org/w/index.php?diff=8833...,Afria Nuswantara


In [5]:
data.tail()

,action,change_size,flags,geo_ip,hashtags,is_anon,is_bot,is_minor,is_new,is_unpatrolled,mentions,ns,page_title,parent_rev_id,rev_id,summary,timestamp,url,user
7944,edit,-6.0,None,NaN,[],False,False,False,False,False,[],Template,Template:New Jersey State Senators,883364554.0,880713885.0,None,2019-02-14 23:43:53.182064,https://en.wikipedia.org/w/index.php?diff=8833...,Muboshgu
7945,edit,77.0,M,NaN,[],False,False,True,False,False,[],Main,Paulo Gazzaniga,883364555.0,882837596.0,None,2019-02-14 23:43:55.498175,https://en.wikipedia.org/w/index.php?diff=8833...,Car rush 1
7946,edit,30.0,N,NaN,[],False,False,False,True,False,[],Main,Inun-onan,NaN,883364557.0,[[WP:AES|←]]Redirected page to [[Paksiw#Inun-u...,2019-02-14 23:43:55.966439,https://en.wikipedia.org/w/index.php?oldid=883...,Obsidian Soul
7947,edit,-1.0,None,"{'city': 'Miami', 'country_name': 'United Stat...",[],True,False,False,False,False,[],Main,W. A. Martin,883364556.0,803817668.0,None,2019-02-14 23:43:56.318894,https://en.wikipedia.org/w/index.php?diff=8833...,2601:586:4104:6907:1E1:4F96:64E6:D917
7948,edit,7.0,MB,NaN,[],False,True,True,False,False,[],Main,Bruce Mayrock,883364558.0,878429616.0,Bot: Fixing double redirect to [[Nigerian Civi...,2019-02-14 23:43:57.136671,https://en.wikipedia.org/w/index.php?diff=8833...,EmausBot


The dataset was collected over an hour long period from approximately 22:43-23:43 14th February 2019.

In [6]:
print("Starting timestamp", data.timestamp.min())
print("Ending timestamp", data.timestamp.max())

Starting timestamp 2019-02-14 22:43:57.810307
Ending timestamp 2019-02-14 23:43:57.136671


In [7]:
# TODO: show edits to English language wikipedia articles only

In [8]:
# only consider edits to main articles submitted by people rather than bots
main_articles = data[(data.ns == "Main") & (data.is_bot == False)]
main_articles.page_title.value_counts()

Alexis Ohanian                                                    24
Miles Graye                                                       16
Sam Humphrey                                                      16
Distinguished Service Medal (United States Navy)                  15
West Michigan Ironmen                                             14
List of theorems                                                  14
2018–19 FC Shakhtar Donetsk season                                14
Epidural hematoma                                                 13
Dumbo (2019 film)                                                 12
England national rugby union team                                 11
Liz MacDonald                                                     10
Plato                                                             10
Greater Khorasan                                                  10
Die Kitty Die                                                     10
Erica Luttrell                    

In [9]:
# go into detail about changes without a summary field provided
main_articles.summary.isnull().value_counts()

False    3577
True      892
Name: summary, dtype: int64

In [10]:
edits_with_summary = main_articles[~main_articles.summary.isnull()]

In [11]:
edits_with_summary.summary

5                                         /* Passenger */
6                                      /* Bibliography */
7                                        /* Boundaries */
11                                         /* Ancestry */
12                           /* References */ {{reflist}}
16                                         /* Language */
22                                         /* Episodes */
25      /* The Empire Strikes Back: A Posttranssexual ...
32                                        Create redirect
34                               /* Prime minister */ ALP
36                                     /* Transfers in */
38                                     /* top */ DAB link
39                                       /* Boundaries */
41                                     Removed propaganda
44       [[WP:AES|←]]Removed redirect to [[Trina (name)]]
45      Change the word “God” to “Allah” because that ...
46      clean up, [[WP:AWB/T|typo(s) fixed]]: fifteen ...
48            

In [12]:
# parse summary
# * remove text within "/* */"
# * remove text within "[]"

In [13]:
def parse_summary(summary):
    
    summary = re.sub("[\(\[\/\{].*?[\}\/\)\]]", "", summary).strip()
    summary = " ".join(re.findall("[a-zA-Z]+", summary))
    summary = summary.lower()

    return summary

In [14]:
edits_with_summary["parsed_summary"] = edits_with_summary.summary.apply(parse_summary)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
edits_with_summary.head(100)

,action,change_size,flags,geo_ip,hashtags,is_anon,is_bot,is_minor,is_new,is_unpatrolled,mentions,ns,page_title,parent_rev_id,rev_id,summary,timestamp,url,user,parsed_summary
5,edit,-50.0,None,NaN,[],False,False,False,False,False,[],Main,Cairo International Airport,883357328.0,882452428.0,/* Passenger */,2019-02-14 22:43:58.774362000,https://en.wikipedia.org/w/index.php?diff=8833...,JCC199,
6,edit,23.0,None,NaN,[],False,False,False,False,False,[],Main,John VIII bar Abdoun,883357334.0,883357286.0,/* Bibliography */,2019-02-14 22:43:58.836848000,https://en.wikipedia.org/w/index.php?diff=8833...,Fadesga,
7,edit,102.0,None,NaN,[],False,False,False,False,False,[],Main,Bootle (UK Parliament constituency),883357332.0,858461701.0,/* Boundaries */,2019-02-14 22:43:58.857889000,https://en.wikipedia.org/w/index.php?diff=8833...,The-,
11,edit,-9.0,None,NaN,[],False,False,False,False,False,[],Main,"Jean, Count of Paris",883357337.0,882798060.0,/* Ancestry */,2019-02-14 22:44:01.906388000,https://en.wikipedia.org/w/index.php?diff=8833...,Alixsommerlath94,
12,edit,-2.0,None,NaN,[],False,False,False,False,False,[],Main,Ralph Santolla,883357338.0,874683436.0,/* References */ {{reflist}},2019-02-14 22:44:01.906413000,https://en.wikipedia.org/w/index.php?diff=8833...,Jax 0677,
16,edit,-4.0,None,NaN,[],False,False,False,False,False,[],Main,Wongkumara,883357342.0,883353579.0,/* Language */,2019-02-14 22:44:03.952624000,https://en.wikipedia.org/w/index.php?diff=8833...,Kwamikagami,
22,edit,1558.0,None,"{'city': 'Williamstown', 'country_name': 'Unit...",[],True,False,False,False,False,[],Main,3Below: Tales of Arcadia,883357347.0,882638905.0,/* Episodes */,2019-02-14 22:44:07.639533000,https://en.wikipedia.org/w/index.php?diff=8833...,172.79.38.250,
25,edit,-8.0,None,NaN,[],False,False,False,False,False,[],Main,Sandy Stone (artist),883357349.0,874894675.0,/* The Empire Strikes Back: A Posttranssexual ...,2019-02-14 22:44:08.978120000,https://en.wikipedia.org/w/index.php?diff=8833...,Autarch,rm wp peacock
32,edit,136.0,N,NaN,[],False,False,False,True,False,[],Main,Tatsumi (List of Akame ga Kill!),NaN,883357351.0,Create redirect,2019-02-14 22:44:13.009506000,https://en.wikipedia.org/w/index.php?oldid=883...,TAnthony,create redirect
34,edit,540.0,None,NaN,[],False,False,False,False,False,[],Main,Émile Combes,883357355.0,878392637.0,/* Prime minister */ ALP,2019-02-14 22:44:16.649415000,https://en.wikipedia.org/w/index.php?diff=8833...,Rjensen,alp


In [16]:
def parse_and_tokenize(summary):
    
    # parse summary
    summary = re.sub("[\(\[\/\{].*?[\}\/\)\]]", "", summary).strip()
    summary = " ".join(re.findall("[a-zA-Z]+", summary))
    summary = summary.lower()

    # get verbs only
    is_verb = lambda pos: pos[:2] == "VB"
    tokenized = nltk.word_tokenize(summary)
    verbs = [word for (word, pos) in nltk.pos_tag(tokenized) if is_verb(pos)]
    
    return verbs
    

In [24]:
edits_with_summary["verbs"] = edits_with_summary.summary.apply(parse_and_tokenize)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
edits_with_summary.head(100)

,action,change_size,flags,geo_ip,hashtags,is_anon,is_bot,is_minor,is_new,is_unpatrolled,...,ns,page_title,parent_rev_id,rev_id,summary,timestamp,url,user,parsed_summary,verbs
5,edit,-50.0,None,NaN,[],False,False,False,False,False,...,Main,Cairo International Airport,883357328.0,882452428.0,/* Passenger */,2019-02-14 22:43:58.774362000,https://en.wikipedia.org/w/index.php?diff=8833...,JCC199,,[]
6,edit,23.0,None,NaN,[],False,False,False,False,False,...,Main,John VIII bar Abdoun,883357334.0,883357286.0,/* Bibliography */,2019-02-14 22:43:58.836848000,https://en.wikipedia.org/w/index.php?diff=8833...,Fadesga,,[]
7,edit,102.0,None,NaN,[],False,False,False,False,False,...,Main,Bootle (UK Parliament constituency),883357332.0,858461701.0,/* Boundaries */,2019-02-14 22:43:58.857889000,https://en.wikipedia.org/w/index.php?diff=8833...,The-,,[]
11,edit,-9.0,None,NaN,[],False,False,False,False,False,...,Main,"Jean, Count of Paris",883357337.0,882798060.0,/* Ancestry */,2019-02-14 22:44:01.906388000,https://en.wikipedia.org/w/index.php?diff=8833...,Alixsommerlath94,,[]
12,edit,-2.0,None,NaN,[],False,False,False,False,False,...,Main,Ralph Santolla,883357338.0,874683436.0,/* References */ {{reflist}},2019-02-14 22:44:01.906413000,https://en.wikipedia.org/w/index.php?diff=8833...,Jax 0677,,[]
16,edit,-4.0,None,NaN,[],False,False,False,False,False,...,Main,Wongkumara,883357342.0,883353579.0,/* Language */,2019-02-14 22:44:03.952624000,https://en.wikipedia.org/w/index.php?diff=8833...,Kwamikagami,,[]
22,edit,1558.0,None,"{'city': 'Williamstown', 'country_name': 'Unit...",[],True,False,False,False,False,...,Main,3Below: Tales of Arcadia,883357347.0,882638905.0,/* Episodes */,2019-02-14 22:44:07.639533000,https://en.wikipedia.org/w/index.php?diff=8833...,172.79.38.250,,[]
25,edit,-8.0,None,NaN,[],False,False,False,False,False,...,Main,Sandy Stone (artist),883357349.0,874894675.0,/* The Empire Strikes Back: A Posttranssexual ...,2019-02-14 22:44:08.978120000,https://en.wikipedia.org/w/index.php?diff=8833...,Autarch,rm wp peacock,[]
32,edit,136.0,N,NaN,[],False,False,False,True,False,...,Main,Tatsumi (List of Akame ga Kill!),NaN,883357351.0,Create redirect,2019-02-14 22:44:13.009506000,https://en.wikipedia.org/w/index.php?oldid=883...,TAnthony,create redirect,[]
34,edit,540.0,None,NaN,[],False,False,False,False,False,...,Main,Émile Combes,883357355.0,878392637.0,/* Prime minister */ ALP,2019-02-14 22:44:16.649415000,https://en.wikipedia.org/w/index.php?diff=8833...,Rjensen,alp,[]


In [19]:
class Lemmatizer(object):
    
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()

    def __call__(self, verbs):
        return [self.lemmatizer.lemmatize(verb, "v") for verb in verbs]

In [29]:
clf = TfidfVectorizer(tokenizer=Lemmatizer(), lowercase=False, stop_words=set(stopwords.words('english')))

# clf.fit(edits_with_summary.verbs)
clf.fit(edits_with_summary[edits_with_summary.verbs.str.len() != 0].verbs)

word2idf = dict(zip(clf.get_feature_names(), clf.idf_))
sorted_idf = sorted(word2idf.items(), key=lambda x: x[1], reverse=True)

In [30]:
sorted_idf

[('accompany', 7.767918333794338),
 ('accord', 7.767918333794338),
 ('account', 7.767918333794338),
 ('accuse', 7.767918333794338),
 ('acquire', 7.767918333794338),
 ('afghanistan', 7.767918333794338),
 ('ahmed', 7.767918333794338),
 ('alexander', 7.767918333794338),
 ('algebra', 7.767918333794338),
 ('allah', 7.767918333794338),
 ('alt', 7.767918333794338),
 ('alter', 7.767918333794338),
 ('alternative', 7.767918333794338),
 ('amend', 7.767918333794338),
 ('anyways', 7.767918333794338),
 ('archive', 7.767918333794338),
 ('article', 7.767918333794338),
 ('assault', 7.767918333794338),
 ('assert', 7.767918333794338),
 ('assist', 7.767918333794338),
 ('assume', 7.767918333794338),
 ('attend', 7.767918333794338),
 ('attribute', 7.767918333794338),
 ('avoid', 7.767918333794338),
 ('balance', 7.767918333794338),
 ('barbados', 7.767918333794338),
 ('biography', 7.767918333794338),
 ('bore', 7.767918333794338),
 ('bowl', 7.767918333794338),
 ('brave', 7.767918333794338),
 ('build', 7.76791833

In [31]:
sorted(word2idf.items(), key=lambda x: x[1], reverse=False)

[('add', 1.8317022609168343),
 ('revert', 3.224623551524334),
 ('use', 3.251579361512862),
 ('fix', 3.7605851485618667),
 ('remove', 3.995157395699699),
 ('clean', 4.418014246519733),
 ('change', 4.797503868224636),
 ('create', 5.059868132692127),
 ('update', 5.093769684367809),
 ('correct', 5.128861004179079),
 ('repair', 5.3255712984251335),
 ('surname', 5.3255712984251335),
 ('fill', 5.370023060995967),
 ('need', 5.465333240800292),
 ('make', 5.570693756458119),
 ('miss', 5.570693756458119),
 ('move', 5.627852170298067),
 ('see', 5.688476792114502),
 ('link', 5.753015313252073),
 ('edit', 5.822008184739024),
 ('redirect', 5.822008184739024),
 ('e', 5.896116156892746),
 ('import', 5.976158864566282),
 ('say', 5.976158864566282),
 ('shortdesc', 5.976158864566282),
 ('expand', 6.263840937018064),
 ('format', 6.263840937018064),
 ('write', 6.263840937018064),
 ('c', 6.381623972674447),
 ('clarify', 6.381623972674447),
 ('improve', 6.381623972674447),
 ('ref', 6.381623972674447),
 ('repl

In [37]:
edits_with_summary[["add" in x for x in edits_with_summary.verbs]]

,action,change_size,flags,geo_ip,hashtags,is_anon,is_bot,is_minor,is_new,is_unpatrolled,...,ns,page_title,parent_rev_id,rev_id,summary,timestamp,url,user,parsed_summary,verbs
520,edit,1.0,None,"{'city': 'Los Alamos', 'country_name': 'United...",[],True,False,False,False,False,...,Main,Guar gum,883357794.0,881305158.0,Typo fix to add space between words,2019-02-14 22:48:07.654969000,https://en.wikipedia.org/w/index.php?diff=8833...,192.12.184.6,typo fix to add space between words,[add]
542,edit,139.0,None,NaN,[],False,False,False,False,False,...,Main,Weightlifting at the 1955 Pan American Games,883357815.0,827663146.0,add navboxes and category,2019-02-14 22:48:16.278442000,https://en.wikipedia.org/w/index.php?diff=8833...,Zyxw,add navboxes and category,[add]
597,edit,139.0,None,NaN,[],False,False,False,False,False,...,Main,Weightlifting at the 1959 Pan American Games,883357864.0,827662935.0,add navboxes and category,2019-02-14 22:48:39.626633000,https://en.wikipedia.org/w/index.php?diff=8833...,Zyxw,add navboxes and category,[add]
632,edit,139.0,None,NaN,[],False,False,False,False,False,...,Main,Weightlifting at the 1967 Pan American Games,883357899.0,869091666.0,add navboxes and category,2019-02-14 22:48:58.467463000,https://en.wikipedia.org/w/index.php?diff=8833...,Zyxw,add navboxes and category,[add]
661,edit,72.0,None,NaN,[],False,False,False,False,False,...,Main,Diana A,883357924.0,575880972.0,/* top */Add Rcat and shell,2019-02-14 22:49:13.238793000,https://en.wikipedia.org/w/index.php?diff=8833...,TAnthony,add rcat and shell,[add]
668,edit,90.0,None,NaN,[],False,False,False,False,False,...,Main,Tetsuya Tsurugi,883357930.0,458767642.0,/* top */Add Rcat and shell,2019-02-14 22:49:16.492731000,https://en.wikipedia.org/w/index.php?diff=8833...,TAnthony,add rcat and shell,[add]
688,edit,139.0,None,NaN,[],False,False,False,False,False,...,Main,Weightlifting at the 1971 Pan American Games,883357946.0,827662363.0,add navboxes and category,2019-02-14 22:49:25.917040000,https://en.wikipedia.org/w/index.php?diff=8833...,Zyxw,add navboxes and category,[add]
693,edit,53.0,M,NaN,[],False,False,True,False,False,...,Main,SNAP-10A,883357954.0,881495641.0,/* Spacecraft history */ add a link for Haystack,2019-02-14 22:49:30.440038000,https://en.wikipedia.org/w/index.php?diff=8833...,Jhfrontz,add a link for haystack,[add]
723,edit,96.0,None,NaN,[],False,False,False,False,False,...,Main,Weightlifting at the 1975 Pan American Games,883357981.0,883313911.0,add navbox and category,2019-02-14 22:49:49.673072000,https://en.wikipedia.org/w/index.php?diff=8833...,Zyxw,add navbox and category,[add]
738,edit,89.0,None,NaN,[],False,False,False,False,False,...,Main,General Birdler,883357996.0,836663981.0,/* top */Add Rcat and shell,2019-02-14 22:49:58.827329000,https://en.wikipedia.org/w/index.php?diff=8833...,TAnthony,add rcat and shell,[add]


In [38]:
from nltk.corpus import wordnet

def get_word_synonyms_from_sent(word, sent):
    word_synonyms = []
    for synset in wordnet.synsets(word):
        for lemma in synset.lemma_names():
            if lemma in sent and lemma != word:
                word_synonyms.append(lemma)
    return word_synonyms

In [47]:
get_word_synonyms_from_sent("change", clf.get_feature_names())

['alter', 'modify', 'alter', 'convert', 'transfer']